# Portfolio based on MVE 

In [191]:
#Importing Libraries

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import yfinance as yf

In [192]:
#Allows Max Display

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [194]:
#Stocks

tickers = ["MMM","AXP","T","BA","CAT","CSCO","KO", "XOM","GE","GS","HD","IBM","INTC","JNJ"
           ,"JPM","MCD","MRK","MSFT","NKE","PFE","PG","TRV","UNH","VZ","V","WMT","DIS"]

In [195]:
#Different time periods to do the analyis on

start=dt.datetime(2012,10,15)
end=dt.datetime(2022,9,15)

In [196]:
#Loading Stock Market Data

Price = pd.DataFrame()

for t in tickers:
    Price[t] = yf.download(t, start=start,end=end, interval="1d")["Adj Close"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [197]:
#calculating Expected Returns and Covariance Matrix

Re = Price.pct_change()
Re = Re.loc[~(Re==0).all(axis=1)]
#Re= Re.fillna(method="backfill")
Re = np.log(Re+1)
ERe = Re.mean()
CovRe = Re.cov()


In [158]:
#To see Returns, Volatility and Sharpe Ratio Concisely

a= ERe*252
a = pd.DataFrame(a, columns=["Expected Returns"])
b = Re.std() * (252**0.5)
b = pd.DataFrame(b, columns=["Volatility"])
Data = pd.concat([a,b],axis=1)
Data["SR"] = Data["Expected Returns"]/Data["Volatility"]

Data.head()

,Expected Returns,Volatility,SR
MMM,0.052727,0.218428,0.241394
AXP,0.115002,0.294044,0.391106
T,0.027770,0.208729,0.133044
BA,0.091791,0.385209,0.238288
CAT,0.109550,0.282943,0.387180


In [159]:
#Weights of Portfolio according to Mean Varience efficiency

W = np.linalg.inv(CovRe)@ERe

##### Exercise - Effects of putting restrictions on weights ( Weights can't turn negative and each stock can't be more than 20 % ) 

In [160]:
W_1 = np.clip(W,0,None)
W_2 = (W_1/W_1.sum())
W_2 = np.clip(W_2,0,0.2)
W_2 = (W_2/W_2.sum())

In [163]:
#Checking the weights
W_2

array([0.        , 0.        , 0.        , 0.        , 0.08902607,
       0.01820348, 0.00236898, 0.        , 0.        , 0.        ,
       0.08642434, 0.        , 0.        , 0.10611128, 0.10890581,
       0.0712227 , 0.        , 0.14996747, 0.03563454, 0.00358597,
       0.05237675, 0.03109181, 0.16770327, 0.        , 0.07737756,
       0.        , 0.        ])

In [164]:
#Performance before adjustment for negative weights and upper limit of 20% on a stock

#MVE weighting - Average Return
P_MVE_returns = (((ERe@W) * 252 ))*100
print ("Average return is ",P_MVE_returns,'%')

#MVE weighting - Standard Deviation
P_MVE_STD = ((W @ CovRe @ W*252)**0.5) * 100
print ("Standard Deviation is ",P_MVE_STD,'%')

#Sharpe Ratio using MVE weighting
print ("Sharpe Ratio is",P_MVE_returns/P_MVE_STD)

Average return is  247.606802414567 %
Standard Deviation is  157.35526759996532 %
Sharpe Ratio is 1.5735526759996534


In [165]:
#Performance after adjustment

#MVE weighting - Average Return
P_MVE_returns = (((ERe@W_2) * 252 ))*100
print ("Average return is ",P_MVE_returns,'%')

#MVE weighting - Standard Deviation
P_MVE_STD = ((W_2 @ CovRe @ W_2*252)**0.5) * 100
print ("Standard Deviation is ",P_MVE_STD,'%')

#Sharpe Ratio using MVE weighting
print ("Sharpe Ratio is",P_MVE_returns/P_MVE_STD)

Average return is  16.69869567856651 %
Standard Deviation is  17.54363738581931 %
Sharpe Ratio is 0.9518377125181705


##### Exercise - How returns differentiates so much by changing the test data. If the returns are good for test date, then the portfolio will do great and vice-e-versa

In [166]:
#When the choosen test sample date have bad returns

Results=pd.DataFrame([],index=[])
ERe=Re[start:mid_2].mean()
CovRe=Re[start:mid_2].cov()
W=np.linalg.inv(CovRe) @ ERe
W = np.clip(W,0,None)
W = (W/W.sum())
W = np.clip(W,0,0.2)
W = (W/W.sum())

Results.at['avgreturn','estimationsample']=W @ ERe*252
Results.at['volatility','estimationsample']=(W @ CovRe @ W*252)**0.5
Results.at['shaperatio','estimationsample']=Results.at['avgreturn','estimationsample']/Results.at['volatility','estimationsample']

#Period of bad returns
ERetest=Re[mid_2:end].mean()
CovRetest=Re[mid_2:end].cov()
Results.at['avgreturn','testsample']=W @ ERetest*252
Results.at['volatility','testsample']=(W @ CovRetest @ W*252)**0.5
Results.at['shaperatio','testsample']=Results.at['avgreturn','testsample']/Results.at['volatility','testsample']
Results

,estimationsample,testsample
avgreturn,0.189733,-0.140400
volatility,0.176151,0.192665
shaperatio,1.077103,-0.728729


In [167]:
#When the choosen test sample date have good returns

Results=pd.DataFrame([],index=[])
ERe=Re[start:mid_2].mean()
CovRe=Re[start:mid_2].cov()
# construct weights and normalize them so the have desired volatility
W=np.linalg.inv(CovRe) @ ERe
W = np.clip(W,0,None)
W = (W/W.sum())
W = np.clip(W,0,0.2)
W = (W/W.sum())

Results.at['avgreturn','estimationsample']=W @ ERe*252
Results.at['volatility','estimationsample']=(W @ CovRe @ W*252)**0.5
Results.at['shaperatio','estimationsample']=Results.at['avgreturn','estimationsample']/Results.at['volatility','estimationsample']

#Period of good returns
ERetest=Re["2022-7-15":"2022-8-15"].mean()
CovRetest=Re["2022-7-15":"2022-8-15"].cov()
Results.at['avgreturn','testsample']=W @ ERetest*252
Results.at['volatility','testsample']=(W @ CovRetest @ W*252)**0.5
Results.at['shaperatio','testsample']=Results.at['avgreturn','testsample']/Results.at['volatility','testsample']
Results

,estimationsample,testsample
avgreturn,0.189733,1.100477
volatility,0.176151,0.176408
shaperatio,1.077103,6.238263


# Exercise - Trading strategy in which weights are being changed monthly

#### Task 1 (incomplete) - how to do quarterly trading

#### Task 2 (incomplete) - for monthly trading, how to consider months where '1st' day of the month does not exist

In [198]:
InSampleResults=pd.DataFrame([],index=[])
Strategy=pd.DataFrame([],index=[])
Results=pd.DataFrame([],index=[])

for date in Re[dt.datetime(2012,1,1):].index:
    if date.day > 1:
        pass
    else:
        ERe=Re[:date- pd.DateOffset(days=1)].mean()
        CovRe=Re[:date- pd.DateOffset(days=1)].cov()
        W = np.linalg.inv(CovRe) @ ERe
        W = np.clip(W,0,None)
        W = (W/W.sum())
        W = np.clip(W,0,0.20)
        W = (W/W.sum())
        InSampleResults.at[date,'avgreturn']=W @ ERe*252
        InSampleResults.at[date,'volatility']=(W @ CovRe @ W*252)**0.5
        InSampleResults.at[date,'sharperatio']=W @ ERe*252/(W @ CovRe @ W*252)**0.5
        Strategy.at[date,'Returns']=W @ Re.loc[date]

Results.at['avgreturn','test_sample']=Strategy.Returns.mean()*252
Results.at['volatility','test_sample']=(Strategy.Returns.std())*252**0.5
Results.at['shaperatio','test_sample']=Results.at['avgreturn','test_sample']/Results.at['volatility','test_sample']

In [200]:
#Training data result
InSampleResults.mean()

avgreturn      0.202884
volatility     0.138097
sharperatio    1.555398
dtype: float64

In [201]:
#Testing data result
Results

,test_sample
avgreturn,0.218524
volatility,0.157822
shaperatio,1.384619


## <span style='color:green'> Testing sample result is quite good as compared to the training results. Also, getting a return of 21.85% with a volatility of 15.78% is a very good performance of a portfolio. We have outperformed the Nasdaq 100 with our weighting of the Nasdaq 100 stocks </span>

## Exercise (incomplete) - How to account for situations like when there are more than 4 tech stocks, and we want to choose only 1 or 2.

In [229]:
#Correlation data
correlation = Re.corr()

In [230]:
#Performing for loop to get correlation data where correlation is either greater than 0.5 and less than 1.0 and putting it under
# the data frame 'high_correlation_1'

stocks = []
stocks_2 = []
corr = []
high_correlation = pd.DataFrame([])
for i in correlation.columns:
    for m in correlation[i]:
            if m>0.50 and m<=1:
                stocks.append(i)
                s = correlation.loc[correlation[i]==m].index.values
                s = "".join(correlation.loc[correlation[i]==m].index.values)
                s = s.replace("'", "")
                #print (s)
                stocks_2.append(s)
                corr.append(m)
                
stocks = pd.DataFrame(stocks)
stocks = stocks.rename(columns={stocks.columns[0]:"Stock Name 1"})    

stocks_2 = pd.DataFrame(stocks_2)
stocks_2 = stocks_2.rename(columns={stocks_2.columns[0]:"Stock Name 2"})   

corr = pd.DataFrame(corr)
corr = corr.rename(columns={corr.columns[0]:"corr"}) 

In [220]:
high_correlation_1 = pd.concat([stocks,stocks_2,corr],axis=1)

In [225]:
high_correlation_1

,Stock Name 1,Stock Name 2,corr
0,MMM,MMM,1.000000
1,MMM,AXP,0.512661
2,MMM,CAT,0.592726
3,MMM,CSCO,0.517032
4,MMM,GS,0.558350
5,MMM,HD,0.502540
6,MMM,IBM,0.529888
7,MMM,JNJ,0.512528
8,MMM,JPM,0.556603
9,MMM,V,0.503120


In [234]:
high_correlation_2 = correlation.melt(ignore_index=False).set_index("variable", append=True).reorder_levels(["variable", None])

In [244]:
high_correlation_2

value
variable               
MMM      MMM   1.000000
         AXP   0.512661
         T     0.459626
         BA    0.433101
         CAT   0.592726
         CSCO  0.517032
         KO    0.482599
         XOM   0.467740
         GE    0.499060
         GS    0.558350
         HD    0.502540
         IBM   0.529888
         INTC  0.447378
         JNJ   0.512528
         JPM   0.556603
         MCD   0.450262
         MRK   0.362218
         MSFT  0.434735
         NKE   0.442919
         PFE   0.395358
         PG    0.426583
         TRV   0.492226
         UNH   0.435088
         VZ    0.379391
         V     0.503120
         WMT   0.300031
         DIS   0.463553
AXP      MMM   0.512661
         AXP   1.000000
         T     0.448434
         BA    0.610766
         CAT   0.543543
         CSCO  0.461385
         KO    0.485258
         XOM   0.536897
         GE    0.539961
         GS    0.691008
         HD    0.495667
         IBM   0.495738
         INTC  0.440360
         JNJ   0.405977
         JPM   0.738759
         MCD   0.498290
         MRK   0.343359
         MSFT  0.457646
         NKE   0.485233
         PFE   0.351683
         PG    0.346552
         TRV   0.583024
         UNH   0.481297
         VZ    0.322899
         V     0.644230
         WMT   0.228669
         DIS   0.587844
T        MMM   0.459626
         AXP   0.448434
         T     1.000000
         BA    0.386809
         CAT   0.425349
         CSCO  0.408130
         KO    0.495709
         XOM   0.443072
         GE    0.415918
         GS    0.467286
         HD    0.407100
         IBM   0.471505
         INTC  0.361379
         JNJ   0.439780
         JPM   0.497179
         MCD   0.390818
         MRK   0.381394
         MSFT  0.332475
         NKE   0.344082
         PFE   0.370814
         PG    0.454889
         TRV   0.467983
         UNH   0.384952
         VZ    0.664938
         V     0.397464
         WMT   0.326997
         DIS   0.439560
BA       MMM   0.433101
         AXP   0.610766
         T     0.386809
         BA    1.000000
         CAT   0.490169
         CSCO  0.389273
         KO    0.419674
         XOM   0.498678
         GE    0.550754
         GS    0.560623
         HD    0.456135
         IBM   0.444792
         INTC  0.420098
         JNJ   0.319061
         JPM   0.583617
         MCD   0.440729
         MRK   0.258513
         MSFT  0.392765
         NKE   0.432729
         PFE   0.283869
         PG    0.270789
         TRV   0.487565
         UNH   0.385289
         VZ    0.264204
         V     0.500815
         WMT   0.192639
         DIS   0.521045
CAT      MMM   0.592726
         AXP   0.543543
         T     0.425349
         BA    0.490169
         CAT   1.000000
         CSCO  0.462940
         KO    0.375198
         XOM   0.587837
         GE    0.511325
         GS    0.604101
         HD    0.418276
         IBM   0.491538
         INTC  0.435317
         JNJ   0.371336
         JPM   0.609185
         MCD   0.361254
         MRK   0.321276
         MSFT  0.417402
         NKE   0.401976
         PFE   0.328050
         PG    0.288826
         TRV   0.459472
         UNH   0.376946
         VZ    0.321610
         V     0.471311
         WMT   0.237919
         DIS   0.454819
CSCO     MMM   0.517032
         AXP   0.461385
         T     0.408130
         BA    0.389273
         CAT   0.462940
         CSCO  1.000000
         KO    0.412803
         XOM   0.402774
         GE    0.390046
         GS    0.497627
         HD    0.476799
         IBM   0.523398
         INTC  0.526938
         JNJ   0.470246
         JPM   0.499759
         MCD   0.439087
         MRK   0.381389
         MSFT  0.561847
         NKE   0.432252
         PFE   0.397831
         PG    0.430970
         TRV   0.413287
         UNH   0.420162
         VZ    0.346053
         V     0.522376
         WMT   0.345129
         DIS   0.474115
KO       MMM   0.482599
         AXP   0.485258
         T     0.495709
         B

## Exercise (incomplete) - Change in Portfolio weights only when there is a change of more than 2%

In [1533]:
Weights = []

for date in Re[dt.datetime(2012,1,1):].index:
    if date.day > 1:
        pass
    else:
        ERe=Re[:date- pd.DateOffset(days=1)].mean()
        CovRe=Re[:date- pd.DateOffset(days=1)].cov()
        W = np.linalg.inv(CovRe) @ ERe
        W = np.clip(W,0.001,None)
        W = (W/W.sum())
        #W = np.clip(W,0,0.20)
        W = (W/W.sum())
        Weights.append(W)

data = pd.DataFrame()
#for counter, symbol in enumerate(Re.columns):
    #data[symbol+' weight'] = [w[counter] for w in Weights]

In [1534]:
data = pd.DataFrame()
for counter, symbol in enumerate(Re.columns):
    data[symbol+' weight'] = [w[counter] for w in Weights]

In [1535]:
data_change = data.pct_change()

In [1536]:
data_change = data_change.dropna()

In [1624]:
data_change.head()*100

,AAPL weight,NKE weight,GOOGL weight,AMZN weight
1,4.639548,-22.824031,-22.570180,-22.570180
2,-7.742523,22.479425,17.017375,45078.619519
3,5.665871,-6.642228,9.741889,-91.458595
4,11.687799,-99.917170,13.785982,1524.983252
5,1.234821,7.046632,7.046632,-22.124558


In [1553]:
(data_change.loc[(data_change["AAPL weight"] < 0.02) & (data_change["AAPL weight"] > -0.02)])

,AAPL weight,NKE weight,GOOGL weight,AMZN weight
5,0.012348,0.070466,0.070466,-0.221246
6,-0.016217,0.168939,0.168939,0.376079
21,-0.003112,-0.027927,-0.151395,1.693166
22,0.005606,-0.041474,-0.295152,0.911872
36,0.010530,0.075634,-0.075622,-0.022068
39,0.007458,-0.169126,0.213384,0.045846
57,0.009874,-0.047194,2.907914,-0.120986
60,0.006739,-0.210512,-0.997078,0.458102
61,-0.004867,-0.118905,-0.030254,0.070011
69,-0.008015,-0.027886,0.236394,-0.000417


In [1560]:
for i in data_change.columns:
    print (i)

AAPL weight
NKE weight
GOOGL weight
AMZN weight


In [1565]:
for i in data_change.columns:
    df = pd.DataFrame(pd.DataFrame(((data_change.loc[(data_change[i] < 0.02) & (data_change[i] > -0.02)]).head())))

In [1566]:
df

,AAPL weight,NKE weight,GOOGL weight,AMZN weight
24,-0.062305,0.099523,-0.238222,0.011607
34,0.021601,0.037355,-0.074534,-0.007497
37,0.073301,-0.017646,-0.073760,0.004116
41,-0.058778,0.314151,-0.181378,-0.004273
42,0.177482,-0.251584,0.048383,0.005192


In [1567]:
for i in data_change.columns:
    print ((data_change.loc[(data_change[i] < 0.02) & (data_change[i] > -0.02)]).head())

    AAPL weight  NKE weight  GOOGL weight  AMZN weight
5   0.012348     0.070466    0.070466     -0.221246   
6  -0.016217     0.168939    0.168939      0.376079   
21 -0.003112    -0.027927   -0.151395      1.693166   
22  0.005606    -0.041474   -0.295152      0.911872   
36  0.010530     0.075634   -0.075622     -0.022068   
    AAPL weight  NKE weight  GOOGL weight  AMZN weight
18  0.082219    -0.000760   -0.018560     -0.561083   
37  0.073301    -0.017646   -0.073760      0.004116   
49 -0.072888    -0.006134    0.308638     -0.016568   
55  0.029592     0.008672   -0.224732      0.031849   
56  0.062116     0.008543   -0.729177      0.098404   
    AAPL weight  NKE weight  GOOGL weight  AMZN weight
17  0.053612    -0.026193    0.006458     -0.217265   
18  0.082219    -0.000760   -0.018560     -0.561083   
30  0.229759    -0.107457    0.015183      0.041939   
54  0.097893    -0.090187   -0.014493      0.028893   
59  0.117098    -0.101984    0.017192     -0.008335   
    AAPL w